In [2]:
import pandas as pd

In [5]:
df = pd.read_csv('data1.csv')
df

,Unnamed: 0,motion_sensor1,motion_sensor2,motion_sensor3,motion_sensor4,motion_sensor5,motion_sensor6,camera1,camera2,camera3,...,r28,r29,r30,r31,r32,r33,r34,c1,c2,outside
0,0,no motion,no motion,no motion,no motion,motion,motion,0,0,0,...,0,0,0,0,0,0,0,0,0,40
1,1,no motion,motion,no motion,no motion,motion,no motion,0,0,0,...,0,0,0,0,0,0,0,0,0,40
2,2,no motion,no motion,no motion,motion,no motion,no motion,0,0,0,...,0,0,0,0,0,0,0,1,0,39
3,3,no motion,motion,no motion,no motion,no motion,no motion,0,0,0,...,0,0,0,0,0,0,0,0,0,37
4,4,no motion,motion,no motion,no motion,no motion,no motion,0,0,0,...,0,0,0,0,0,0,0,1,0,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,2395,no motion,no motion,no motion,no motion,no motion,no motion,0,0,0,...,0,0,0,0,0,0,0,0,0,41
2396,2396,no motion,no motion,no motion,motion,no motion,no motion,0,0,0,...,0,0,0,0,0,0,0,0,0,41
2397,2397,no motion,motion,no motion,no motion,no motion,no motion,0,0,0,...,0,0,0,0,0,0,0,0,0,41
2398,2398,no motion,no motion,no motion,motion,no motion,motion,0,0,0,...,0,0,0,0,0,0,0,0,0,41


In [6]:
room1_sensor1 = df[['r1', 'motion_sensor1']]
room1_sensor1

,r1,motion_sensor1
0,0,no motion
1,0,no motion
2,0,no motion
3,0,no motion
4,0,no motion
...,...,...
2395,0,no motion
2396,0,no motion
2397,0,no motion
2398,0,no motion


In [7]:
room1_sensor1['r1'] = room1_sensor1['r1'] > 0
room1_sensor1

/tmp/ipykernel_53097/4133476929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  room1_sensor1['r1'] = room1_sensor1['r1'] > 0


,r1,motion_sensor1
0,False,no motion
1,False,no motion
2,False,no motion
3,False,no motion
4,False,no motion
...,...,...
2395,False,no motion
2396,False,no motion
2397,False,no motion
2398,False,no motion


In [16]:
(room1_sensor1['r1'] & (room1_sensor1['motion_sensor1']=='motion')).unique()

array([False,  True])

In [17]:
both = (room1_sensor1['r1'] & (room1_sensor1['motion_sensor1']=='motion')).sum()
r1people= (room1_sensor1['r1'] ).sum()

#prob motion sensor is on given people in room 1
both/r1people

np.float64(0.9494479953515398)

In [11]:
both

r1                0
motion_sensor1    0
dtype: object